# Clean Air Act Formal Enforcements by Congressional District

This notebook examines ECHO data on formal enforcement cases, the next step after inspections by which the EPA and state agencies move forward to enforce environmental laws.  It uses the following downloadable ECHO spreadsheets:  CASE_FACILITIES, CASE_ENFORCEMENTS and ECHO_EXPORTER.
The fields in the CASE_FACILITIES table are:
<ol>
    <li>REGISTRY_ID, </li>
    <li>CASE_NUMBER. </li>
</ol>
The fields in the CASE_ENFORCEMENTS table are:
<ol>
    <li>CASE_NUMBER, </li>
    <li>FISCAL_YEAR, </li>
    <li>CASE_STATUS_DATE, </li>
    <li>HQ_DIVISION. </li>
</ol>
The CASE_NUMBER field links these tables.

The REGISTRY_ID identifier is linked to the ECHO_EXPORTER table's field of the same name.

The HQ_DIVISION of interest for this notebook is 'CAA'.
    
**A state and congressional district must be chosen using the dropdown
widgets that are provided.**

Note:  This notebook has widgets to select the state, congressional district and facility.  Those currently work only when you **run cells one at a time sequentially, not all at once.**

---
---

## How to Run
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Some cells, like the one shown below, will create a dropdown menu after you run them. Be sure to make a selection (for example, click to change NY to LA) before running the next cell.
![Dropdown menu](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/dropdown.JPG?raw=true)
* Other cells will simply print information when you run them, like this one:
![Simple cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/cell-simple.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

In [1]:
---
---

SyntaxError: invalid syntax (<ipython-input-1-667f777f690f>, line 1)

# Let's Begin!

### Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

In [2]:
# Import libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


### Run this next cell to create the widget for selecting states. It will create a dropdown menu at the bottom. Choose your state from the menu then move on to the next cell.

In [3]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
dropdown_state=widgets.Dropdown(
    options=states,
    value='NY',
    description='State:',
    disabled=False,
)
output_state = widgets.Output()
my_state = ""

def dropdown_state_eventhandler( change ):
    output_state.clear_output()
    value = change.new
    with output_state:
        display( change.new )
            
dropdown_state.observe( dropdown_state_eventhandler, names='value')
display( dropdown_state )

Dropdown(description='State:', index=32, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', …

### Run this cell after choosing a state. It will pull the data for that state from ECHO

In [4]:
my_state = dropdown_state.value

sql = "select REGISTRY_ID, FAC_NAME, FAC_STATE, FAC_LAT, FAC_LONG, FAC_DERIVED_CD113 " + \
    " from ECHO_EXPORTER where AIR_FLAG = 'Y' and FAC_STATE = '" + my_state + "'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
print(sql)
print(data_location)


select REGISTRY_ID, FAC_NAME, FAC_STATE, FAC_LAT, FAC_LONG, FAC_DERIVED_CD113  from ECHO_EXPORTER where AIR_FLAG = 'Y' and FAC_STATE = 'NY'
http://apps.tlt.stonybrook.edu/echoepa/?query=select%20REGISTRY_ID%2C%20FAC_NAME%2C%20FAC_STATE%2C%20FAC_LAT%2C%20FAC_LONG%2C%20FAC_DERIVED_CD113%20%20from%20ECHO_EXPORTER%20where%20AIR_FLAG%20%3D%20%27Y%27%20and%20FAC_STATE%20%3D%20%27NY%27


### Run this cell to load the CSV of that data. Then, we will be able to analyze it!

In [5]:

echo_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
echo_data.set_index( "REGISTRY_ID", inplace=True)


### How many facilities in the selected state are tracked under the Clean Air Act in the ICIS database?

In [6]:
num_facilities = echo_data.shape[0]
print("There are %s ICIS facilities in %s tracked in the ECHO database." %(num_facilities, my_state))

There are 12464 ICIS facilities in NY tracked in the ECHO database.


### Run this next cell to generate the Congressional District dropdown list for your state.  Then select a CD and run the following cell.
#### Here is a map of congressional districts: https://www.govtrack.us/congress/members/map

In [7]:
if (( my_state != 'none' ) & (my_state != 'all' )):
    cd_array = echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int).unique()
    cd_array.sort()
    w2=widgets.Dropdown(
        options=cd_array,
        value=1,
        description='Congressional Districts:',
        disabled=False,
    )
    display(w2)

Dropdown(description='Congressional Districts:', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…

### Run this next cell after choosing a congressional district from the dropdown list.
#### You will then see the total number of ICIS (EPA's Integrated Compliance System) facilities that are being regulated in your district under the Clean Air Act.

In [8]:
my_cd = w2.value
my_cd_facs = echo_data[echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int) == my_cd]
num_facilities = my_cd_facs.shape[0]    
print("There are %s ICIS facilities in %s district %s tracked in the ECHO database." %(num_facilities, my_state, my_cd))

There are 400 ICIS facilities in NY district 1 tracked in the ECHO database.


### Next run the next few cells to look up the formal enforcement history for the facilities in the selected state and congressional district.  This step may take a while to run.

In [9]:
# sql = "select PGM_SYS_ID, STATE_EPA_FLAG, AGENCY_TYPE_DESC, COMP_MONITOR_DESC, ACTUAL_END_DATE from `ICIS-AIR_FCES_PCES` where STATE_CODE='" + my_state + "'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='

sql = "select CF.REGISTRY_ID, CE.FISCAL_YEAR, CE.CASE_STATUS_DATE from `CASE_FACILITIES` CF, `CASE_ENFORCEMENTS` CE" + \
   " where CE.HQ_DIVISION = 'AIR' and CE.CASE_NUMBER = CF.CASE_NUMBER"
data_location=url+urllib.parse.quote(sql)
print(sql)
print(data_location)

select CF.REGISTRY_ID, CE.FISCAL_YEAR, CE.CASE_STATUS_DATE from `CASE_FACILITIES` CF, `CASE_ENFORCEMENTS` CE where CE.HQ_DIVISION = 'AIR' and CE.CASE_NUMBER = CF.CASE_NUMBER
http://apps.tlt.stonybrook.edu/echoepa/?query=select%20CF.REGISTRY_ID%2C%20CE.FISCAL_YEAR%2C%20CE.CASE_STATUS_DATE%20from%20%60CASE_FACILITIES%60%20CF%2C%20%60CASE_ENFORCEMENTS%60%20CE%20where%20CE.HQ_DIVISION%20%3D%20%27AIR%27%20and%20CE.CASE_NUMBER%20%3D%20CF.CASE_NUMBER


In [10]:
icis_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
icis_data.set_index( "REGISTRY_ID", inplace=True)

In [11]:
# Find the facility that matches the inspection, by REGISTRY_ID.

my_cd_icis = pd.DataFrame()
no_data_ids = []
# Look through all facilities in my CD.
for fac in my_cd_facs.itertuples():
    # breakpoint()
    fac_id = fac.Index
    try:
        enforcements = icis_data.loc[ fac_id ].copy()
        # breakpoint()
        n = enforcements.shape[0]
        fac_list = [fac.Index] * n
        enforcements['facility'] = fac_list
        frames = [my_cd_icis, enforcements]
        my_cd_icis = pd.concat( frames, ignore_index=False )
    except KeyError:
        # The facility wasn't found in the ICIS_FEC_EPA_INSPECTIONS data.
        no_data_ids.append( fac.Index )
    
# my_cd_icis.to_csv( r'my_cd_enforcements.csv', index=True, header=True )
# with open( "no_data_ids.txt", "w" ) as output:
#     output.write( str( no_data_ids ))

### How many facilities have received enforcement actions?

#### The table that follows lists the ID, date, and facility ID of each formal enforcement action tracked in the database.

In [12]:

print( "Facilities without enforcements: " + str( len( no_data_ids )))
print( "Facilities with enforcements: " + str( len( my_cd_icis )))
my_cd_icis

Facilities without enforcements: 397
Facilities with enforcements: 30


,FISCAL_YEAR,CASE_STATUS_DATE,facility
REGISTRY_ID,,,
110001585990,1986,11/17/1987,1.100016e+11
110001585990,1986,11/17/1987,1.100016e+11
110000616726,1998,02/24/1998,1.100006e+11
110000616726,1998,02/24/1998,1.100006e+11
110000616726,1998,02/24/1998,1.100006e+11
110000616726,1998,02/24/1998,1.100006e+11
110000616726,1998,02/24/1998,1.100006e+11
110000616726,1998,02/24/1998,1.100006e+11
110000616726,1998,02/24/1998,1.100006e+11


## This section saves some of this data to a CSV file in your Google Drive.
The first of the next three cells will open our Google Drive to write into.
The second cell writes the congressional district file.
The third cell writes the file for state data.
**Running these cells is optional.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Write the congressional district data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/cd_air-enforce-' + my_state + '-' + str( my_cd ) + '.csv'
my_cd_icis.to_csv( filename ) 
print( "Writing this data to %s" %(filename))

Write the state data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/state-air-enforce-' + my_state + '.csv'
icis_data.to_csv( filename ) 
print( "Writing this data to %s" %(filename))

## Make an interactive map of facilities that have received enforcement actions
### Once you run this cell, a map should appear. You can zoom in and out, or click on facilities to get their names.

In [14]:
# Let's show a quick map of your area and the facilities in it

def mapper(df):
    # Initialize the map
    m = folium.Map(
        location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]],
        zoom_start = 11
    )

    # Add a clickable marker for each facility
    for index, row in df.iterrows():
        folium.Marker(
            location = [row["FAC_LAT"], row["FAC_LONG"]],
            popup = row["FAC_NAME"] ).add_to(m)

    # Show the map
    return m

fac_idx_list = my_cd_icis['facility'].dropna()
fac_cd_icis = my_cd_facs.loc[fac_idx_list]

map_of_facilities_in_my_area = mapper(fac_cd_icis)
map_of_facilities_in_my_area

## Enforcement actions by facility
### Run the below cell, then choose a facility from the dropdown that appears to delve deeper into enforcement actions at that facility.

In [ ]:
cd_array = fac_cd_icis["FAC_NAME"].unique()
cd_array.sort()

In [ ]:
w3=widgets.Dropdown(
    options=cd_array,
    description='Facility Name:',
    disabled=False,
)
display(w3)

Run this next cell after choosing a facility.

In [ ]:
my_fac = fac_cd_icis[fac_cd_icis["FAC_NAME"] == w3.value]
enforcements = my_cd_icis[my_cd_icis['facility'] == my_fac.iloc[[0]].index[0]]

## Plot the number of enforcements by year for the congressional district.

In [ ]:
# This cell creates a function that will be used by both the CD and the state
# to plot the number of cases by year.
import datetime

def show_plot( df, date_field, year_field, place, date_format, chart_title ):
    format_str = date_format # The format
    nan_count = 0
    year_col = []
    for day in df[date_field]:
        try:
            # breakpoint()
            viol_year = datetime.datetime.strptime(day, format_str).year
            year_col.append( viol_year )
        except:
            nan_count += 1
            year_col.append(  np.NaN )
    df[year_field] = year_col
    
    year_groups = df.groupby( year_field )[[ year_field ]]
    counted_years = year_groups.count()

    # Print how many values are present 
    print(counted_years)
    chart_title +=  " in " + place + " by year"

    ax = counted_years[[year_field]].plot(kind='bar', title = chart_title, figsize=(15, 10), legend=False, fontsize=12)
    ax.set_xlabel("Year", fontsize=12)
    ax.set_ylabel("Count", fontsize=12)

chart_title = "Total CAA enforcements"

show_plot( my_cd_icis, 'CASE_STATUS_DATE', 'CASE_STATUS_YEAR', \
          my_state + ' - #' + str( my_cd ), "%m/%d/%Y", chart_title )

## Plot the number of enforcements by year, using the entire state.
Since the number of enforcements in a single CD may be small, it can be more
interesting to look at the entire state.

In [ ]:
chart_title = "Total CAA enforcements"

show_plot( icis_data, 'CASE_STATUS_DATE', 'CASE_STATUS_YEAR', my_state, \
          "%m/%d/%Y", chart_title )